In [85]:
import pandas as pd
from IPython.display import HTML, clear_output


df = pd.read_json('regen.json')
df.head(3)

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
0,Create a modern and elegant website for an onl...,"<!DOCTYPE html>\n<html lang=""en"">\n<head>\n <...",{'Create a website for an online magazine dedi...,0.302043,1,
1,Design an interactive and user-engaging websit...,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...",{'Create an interactive portfolio website that...,0.308442,1,
2,Develop a website for a travel agency that foc...,"<!DOCTYPE html>\n<html lang=""en"">\n<head>\n <...",{'Create a modern and elegant website for an o...,0.341845,1,


In [86]:
curation_df = pd.DataFrame(columns=['curation_status', 'curation_message'])

In [ ]:
for idx, row in df[df['curation_status'] == 0].iterrows():
  last_idx = idx
  instruction, output, most_similar_instructions, avg_similarity_score, _, __ = row
  stop = False
  while not stop:
    clear_output(wait=True)
    display(HTML(output))
    
    print(f'''Instruction: {instruction}
    Output: {output}''')
    
    curator_input = input(f'{idx}. Is the data good? (yes/no/mark/stop):')
    curator_input = curator_input.strip()
    if curator_input == 'yes':
      curation_df.loc[idx] = {
        'curation_status': 1,
        'curation_message': '',
      }
      break
    elif curator_input == 'no':
      curation_df.loc[idx] = {
        'curation_status': -1,
        'curation_message': '',
      }
      break
    # Mark the data if you think the data can be fixed/improved manually
    elif curator_input == 'mark':
      suggestion = input('Suggestion: ')
      curation_df.loc[idx] = {
        'curation_status': 0,
        'curation_message': suggestion,
      }
      break
    elif curator_input == 'stop':
      stop = True
    else:
      print('Invalid input:', '"' + curator_input + '"')
  if stop:
    break


In [88]:
curation_df

,curation_status,curation_message
13,1,


In [92]:
curated_df = curation_df.combine_first(df)[df.columns]
curated_df.iloc[curation_df.index]

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
13,Create a website for a photography portfolio t...,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\...",{'Create a website for an online store that se...,0.352588,1,


In [93]:
curated_df[curated_df['curation_status'] != -1].to_json('regen-curated.json', orient="records")

In [ ]:
df[['instruction', 'output']].to_json('webgen-dataset.json', orient='records')
print('webgen-dataset.json exported sucessfully!')